# **Libraries & Constants**

In [54]:
import pandas as pd
import tensorflow as tf
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Input, Softmax, BatchNormalization, Dropout, Conv2D, MaxPool2D, Embedding, Reshape, Flatten, LSTM, GRU, SimpleRNN, Concatenate, concatenate, RepeatVector, TimeDistributed, Bidirectional
# from keras.layers.merge import concatenate
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax, Nadam, SGD
from tensorflow.keras import Model
from tensorflow.keras.optimizers.schedules import InverseTimeDecay, ExponentialDecay
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences #from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn import metrics
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
import os
import string
from google.colab import drive
import json

from nltk.tokenize import WhitespaceTokenizer
from nltk.stem.porter import PorterStemmer
import spacy
import nltk
nltk.download('stopwords')
import re
# from transformers import AutoTokenizer,BertTokenizer,TFBertModel

import copy

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Load Dataset and declare paths**

In [ ]:
drive.mount('/content/drive')
dataPath = '/content/drive/MyDrive/AI Projects/Sentiment-Analysis/amazon_reviews_us_Watches_v1_00.csv'

# **Preprocessing**

In [55]:
dataset = pd.read_csv(dataPath)

<ipython-input-55-5118f10c515a>:1: DtypeWarning: Columns (17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(dataPath)


In [56]:
useless_features = ['review_date'] + list(dataset.columns[8:11]) + list(dataset.columns[0:7]) + list(dataset.columns[-7:])
print(useless_features)

['review_date', 'helpful_votes', 'total_votes', 'vine', 'marketplace', 'customer_id', 'review_id', 'product_id', 'product_parent', 'product_title', 'product_category', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21']


In [57]:
dataset.drop(columns=useless_features, inplace=True)

In [58]:
dataset.head()

,star_rating,verified_purchase,review_headline,review_body
0,5,Y,Five Stars,Absolutely love this watch! Get compliments al...
1,5,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.
2,2,Y,Two Stars,Scratches
3,5,Y,Five Stars,"It works well on me. However, I found cheaper ..."
4,4,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...


In [59]:
star_rating = np.array(dataset['star_rating'])
review_headline = np.array(dataset['review_headline'])
review_body = np.array(dataset['review_body'])

In [60]:
print(np.count_nonzero(star_rating>=4))

743979


In [61]:
def w_tokenizer(text):
  tokenizer = WhitespaceTokenizer()
  tokenized_list = tokenizer.tokenize(text)
  return tokenized_list

def stemmer_porter(text_list):
  porter = PorterStemmer()
  return_list = []
  for i in range(len(text_list)):
      return_list.append(porter.stem(text_list[i]))
  return(return_list)


def remove_stopwords(text_list):
  NLTK_stopwords = nltk.corpus.stopwords.words('english')
  return_list = []
  for i in range(len(text_list)):
      if text_list[i] not in NLTK_stopwords:
          return_list.append(text_list[i])
  return return_list

def remove_punc(text):
    if isinstance((text), (str)):
        text = re.sub('<[^>]*>', '', text)
        text = re.sub('[\W]+', '', text.lower())
        return text
    if isinstance((text), (list)):
        return_list = []
        for i in range(len(text)):
            temp_text = re.sub('<[^>]*>', '', text[i])
            temp_text = re.sub('[\W]+', '', temp_text.lower())
            return_list.append(temp_text)
        return(return_list)
    else:
        pass

def preprocess_sentence(sentence):
  return remove_punc(remove_stopwords(stemmer_porter(w_tokenizer(sentence))))

In [62]:
print(review_body[0])
print(preprocess_sentence(review_body[0]))

Absolutely love this watch! Get compliments almost every time I wear it. Dainty.
['absolut', 'love', 'thi', 'watch', 'get', 'compliment', 'almost', 'everi', 'time', 'wear', 'it', 'dainty']
